In [1]:
import pyaudio
import struct


ModuleNotFoundError: No module named 'pyaudio'

In [ ]:
import math

FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100

p = pyaudio.PyAudio()

In [ ]:



def data_for_freq(frequency: float, time: float = None):
    """get frames for a fixed frequency for a specified time or
    number of frames, if frame_count is specified, the specified
    time is ignored"""
    frame_count = int(RATE * time)

    remainder_frames = frame_count % RATE
    wavedata = []

    for i in range(frame_count):
        a = RATE / frequency  # number of frames per wave
        b = i / a
        # explanation for b
        # considering one wave, what part of the wave should this be
        # if we graph the sine wave in a
        # displacement vs i graph for the particle
        # where 0 is the beginning of the sine wave and
        # 1 the end of the sine wave
        # which part is "i" is denoted by b
        # for clarity you might use
        # though this is redundant since math.sin is a looping function
        # b = b - int(b)

        c = b * (2 * math.pi)
        # explanation for c
        # now we map b to between 0 and 2*math.PI
        # since 0 - 2*PI, 2*PI - 4*PI, ...
        # are the repeating domains of the sin wave (so the decimal values will
        # also be mapped accordingly,
        # and the integral values will be multiplied
        # by 2*PI and since sin(n*2*PI) is zero where n is an integer)
        d = math.sin(c) * 32767
        e = int(d)
        wavedata.append(e)

    for i in range(remainder_frames):
        wavedata.append(0)

    number_of_bytes = str(len(wavedata))  
    wavedata = struct.pack(number_of_bytes + 'h', *wavedata)

    return wavedata


def play(frequency: float, time: float):
    """
    play a frequency for a fixed time!
    """
    frames = data_for_freq(frequency, time)
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, output=True)
    stream.write(frames)
    stream.stop_stream()
    stream.close()


if __name__ == "__main__":
    play(400, 1)

In [ ]:
import sympy as s
from scipy.linalg import solve_continuous_are
from numerics.utilities.misc import *


[kappa0, eta0, omega0, xi0], [kappa1, eta1, omega1, xi1] = params
model_cte = np.sqrt(2) ### measurement model

def give_matrices(kappa, eta, omega, xi):
    A = np.array([[-kappa/2 -xi, omega],[-omega, xi -kappa/2]])
    B = E = -np.sqrt(eta*kappa)*np.array([[1.,0.],[0.,0.]]) #homodyne
    D = np.diag([kappa]*2)
    C = -B.T
    G = E.T
    return A, C, D,G

A1, C1, D1, G1 = give_matrices(kappa1, eta1, omega1, xi1)
A0, C0, D0, G0 = give_matrices(kappa0, eta0, omega0, xi0)

sst1 = solve_continuous_are( (A1-np.dot(G1.T,C1)).T, C1.T, D1 - np.dot(G1.T, G1), np.eye(2)) #### A.T because the way it's implemented!
sst0 = solve_continuous_are( (A0-np.dot(G0.T,C0)).T, C0.T, D0 - np.dot(G0.T, G0), np.eye(2)) #### A.T because the way it's implemented!

XiCov1  = np.dot(sst1, C1.T) + G1.T
XiCov0  = np.dot(sst0, C0.T) + G0.T




gamma = s.Symbol("gamma")
omega = s.Symbol("omega") #freq harmonic oscillator
eta = s.Symbol("eta") #efficiency measurement (appearing in the CPTP decomposiion X,Y)
xi = s.Symbol("xi")

B = E = -s.sqrt(eta*gamma)*np.array([[1,0],[0,0.]]) #homodyne
A = np.array([[-gamma/2 - xi, omega],[-omega, -gamma/2 + xi]])
D = np.diag([gamma]*2)
C = -B.T
G = E.T


varx = s.Symbol("vx")
varp = s.Symbol("vp")
covxp = s.Symbol("cvxp")



cov = s.Matrix([[varx, covxp],[covxp, varp]])
dots = np.array((A - G*C.T)*cov + cov*((A - G*C.T).T) + D-(G*G.T) - (cov*C.T)*(cov*C.T).T)

[dots[0,0], dots[1,1], dots[0,1]]




def ders(sst, params):
    vx, vp, cvxp = sst[0,0], sst[1,1], sst[1,0]
    [gamma, eta, omega, xi] = params
    return [2*cvxp*omega - 1.0*eta*gamma*vx**2 - 1.0*eta*gamma + gamma + 2*vx*(1.0*eta*gamma - gamma/2 - xi),
 -1.0*cvxp**2*eta*gamma - 2*cvxp*omega + gamma + 2*vp*(-gamma/2 + xi),
 -1.0*cvxp*eta*gamma*vx + cvxp*(-gamma/2 + xi) + cvxp*(1.0*eta*gamma - gamma/2 - xi) + omega*vp - omega*vx]


print(ders(sst1, params[1]))

print(ders(sst0, params[0]))